In [ ]:
using LinearAlgebra

t = 0
U = 1
chem_pot = 0
const N = 2
const basis_labels = reshape(Iterators.product([[0,1] for _ in 1:2N]...) .|> collect, (1, 2^(2N)))
const next = [collect(2:N); 1]


In [ ]:
label_to_idx(label::Vector{Integer}) = reduce(+, label .* [2^n for n in 0:length(label)-1]) + 1
label_to_values(label::Vector{Integer}) = [x == label_to_idx(label) ? 1.0 : 0.0 for x in 1:(length(label))^2]
normalize_vector(vec) = [x == 0 ? 0 : 1 for x in vec]

Base.@kwdef struct StateVec
    values::Vector{Float64}
    no_nodes::Integer
    is_zero::Bool = false # distinction - a state vector of all zeros is vacuum, but if is_zero == true, it's treated as a scalar zero
end

StateVec(label::Vector{Integer}) = StateVec(label_to_values(label), div(length(label), 2), false)
StateVec(no_nodes::Integer, is_zero::Bool) = StateVec(zeros(4no_nodes^2), no_nodes, is_zero)
is_superposition(statevec::StateVec) = 2statevec.no_nodes - 1 <= reduce(+, [x == 0 ? 0 : 1 for x in statevec.values])

# pos = 2n - 1 spin up, 2n spin down; n = cell number
function create(statevec::StateVec, pos)
    if statevec.values[pos] == 1 || statevec.is_zero
        return StateVec(statevec.no_nodes, true)
    end
    vals = copy(statevec.values)
    vals[pos] = 1
    return StateVec(vals, statevec.no_nodes, false)
end

function annihilate(statevec::StateVec, pos)
    if statevec.values[pos] == 0 || statevec.is_zero
        return StateVec(statevec.no_nodes, trues)
    end
    vals = copy(statevec.values)
    vals[pos] = 0
    return StateVec(vals, statevec.no_nodes, false)
end

up(cell_no) = Int(2cell_no - 1)
down(cell_no) = Int(2cell_no)

In [ ]:

# H |state>
interacting_term(state, U, N) = 

function interacting_term(statevec::StateVec, basis_labels, U::Float64)
    no_nodes = statevec.no_nodes
    total = zero(statevec.values)
    for (idx, val) in enumerate(statevec.values)
        if val != 0
            state = basis_labels[idx]
            total += U * reduce(+, [state[up(n)] * state[down(n)] for n in 1:no_nodes]) .* label_to_values(state)
        end
    end
    StateVec(total, no_nodes, false)
end

function kinetic_term(statevec::StateVec, t, next)
    no_nodes = statevec.no_nodes
    total = zero(statevec.values)
    for (idx, val) in enumerate(statevec.values)
        tmp1 = annihilate(state, up(next[n]))
        tmp2 = create(tmp1, up(n))
        if typeof(tmp2) != Int
            total[label_to_idx(tmp2)] += 1
        end
        tmp3 = annihilate(state, down(next[n]))
        tmp4 = create(tmp3, down(n))
        if typeof(tmp4) != Int
            total[label_to_idx(tmp4)] += 1
        end
    end
    return t * total
end

# perform H |state> and get statevec of: alpha |state'>
function hamiltonian_on_state(U, t, chem_pot, state, next)
    N = div(length(state), 2)
    # @show state
    interacting = interacting_term(state, U, N)
    # @show interacting
    kinetic = kinetic_term(state, t, next)
    # @show kinetic
    chemical = chem_pot * reduce(+, state) .* label_to_values(state)
    interacting + kinetic - chemical
end

function braket(bra, ket)
#     if normalize_statevec(bra) == normalize_statevec(ket)
        return dot(bra, ket)
#     else
#         return 0.0
#     end
end

# for every H |state> multiply it by every possible <state'|
function get_hamiltonian(basis_labels, U, t, chem_pot, next, N)
    hamiltonian = zeros((2N)^2, (2N)^2)
    right_side = [hamiltonian_on_state(U, t, chem_pot, state, next) for state in basis_labels]
    for i in 1:(2N)^2, j in 1:(2N)^2
        hamiltonian[i, j] = braket(label_to_values(basis_labels[i]), right_side[j])
    end
    hamiltonian
end

In [ ]:
state_vals_to_labels(state_vals) = [x != 0 ? x.* for (idx, x) in enumerate(state_vals)]

In [ ]:
@show bra = label_to_values(basis_labels[13])
@show ket = hamiltonian_on_state(1, 1, 0, basis_labels[10], next)
braket(bra, ket)

In [ ]:
m = get_hamiltonian(basis_labels, U, t, chem_pot, next, N)

In [ ]:
real(eigvals(m))

In [ ]:
using DelimitedFiles
writedlm("matrix, mu=0txt", m)
writedlm("eigenvalues, mu=0.txt", real(eigvals(m)))

In [ ]:
using PlotlyJS
potentials = collect(-4:0.01:4)
eigs_mat = reduce(hcat,[real(eigvals(get_hamiltonian(basis_labels, U, t, pot, next, N))) for pot in potentials])'
eigs_mat = [eigs_mat[:,i] for i in 1:size(eigs_mat,2)]
layout = Layout(
    title="Wartości własne dla różnych potencjałów chemicznych μ, U=t=1",
    xaxis_title="μ",
    yaxis_title="wartości własne hamiltonianu",
)
pl = plot([scatter(x=potentials, y=eig, name="w. własna #$no") for (eig, no) in zip(eigs_mat, 1:(2N)^2)], layout)

In [ ]:
savefig(pl, "wartosci_wlasne.png")
savefig(pl, "wartosci_wlasne.html")

# oblożyć stanami op. liczby cząstek
# model t-J